# My Master-Script

In [1]:
#Jupyter Notebook Importing Imports

import io, os, sys, types

from IPython import get_ipython
import nbformat
from IPython.core.interactiveshell import InteractiveShell

In [2]:
#Find Notebook

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
#Notebook Loader

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.worksheets[0].cells:
            if cell.cell_type == 'code' and cell.language == 'python':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.input)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [4]:
#Notebook Finder

class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
#Final Jupyter Notebook Importing Steps

sys.meta_path.append(NotebookFinder())

ls nbpackage

import nbimp.AdaFruitIOLoginCredentials

SyntaxError: invalid syntax (<ipython-input-5-5bb2728f19e4>, line 5)

In [ ]:
#Coordinates
metadata = {'lat': 36.017433,
           'lon': -84.239815,
           'ele': 0,
           'created_at': None}

In [ ]:
#External Temperature
import smbus

def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [ ]:
#Main Loop Imports

from bme280 import BME280
import ST7735
from enviroplus.noise import Noise
from ltr559 import LTR559
import time

from IPython.display import clear_output
import busio
import board
import adafruit_ccs811

from IPython.display import clear_output
from board import *

In [ ]:
#Main Loop Variables

soundFeed = aio.feeds("sound")
luxFeed = aio.feeds("light")
intTempCfeed = aio.feeds("inttempc")
intTempFfeed = aio.feeds("inttempf")
extemp2Feed = aio.feeds("extemp2")
extempFeed = aio.feeds("extemp")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
tvocsFeed = aio.feeds("new-sensor.tvocs")
co2Feed = aio.feeds("new-sensor.co2")

range=[(100, 1200)]

noise = Noise()
ltr559 = LTR559()
bme280 = BME280()
i2c = board.I2C()
ccs =  adafruit_ccs811.CCS811(i2c)

Amps = noise.get_amplitudes_at_frequency_ranges(range)
Light = ltr559.get_lux()
External = readTemp()
External2 = External * 1.8 + 32
Celcius = bme280.get_temperature()
Fahrenheit = Celcius * 1.8 + 32
InternalC = Celcius - External
InternalF = Fahrenheit - External2
Humidity = bme280.get_humidity()
Pressure = bme280.get_pressure()
CO2 = ccs.eco2
TVOC = ccs.tvoc

time_calculation = time.time()
aio.send_data(co2Feed.key, CO2, metadata)
aio.send_data(tvocsFeed.key, TVOC, metadata)
aio.send_data(soundFeed.key, Amps[0], metadata)
aio.send_data(luxFeed.key, Light, metadata)
aio.send_data(intTempCfeed.key, InternalC, metadata)
aio.send_data(intTempFfeed.key, InternalF, metadata)
aio.send_data(extemp2Feed.key, External2, metadata)
aio.send_data(extempFeed.key, External, metadata)
aio.send_data(humidFeed.key, Humidity, metadata)
aio.send_data(pressFeed.key, Pressure, metadata)
time_elapsed = time.time() - time_calculation

In [ ]:
#Main Loop

while True:
    try:
        Amps = noise.get_amplitudes_at_frequency_ranges(range)
        Light = ltr559.get_lux()
        External = readTemp()
        External2 = External * 1.8 + 32
        InternalC = Celcius - External
        InternalF = Fahrenheit - External2
        Humidity = bme280.get_humidity()
        Pressure = bme280.get_pressure()
        CO2 = ccs.eco2
        TVOCs = ccs.tvoc
        output = "Temperature: External: {0:.2f} C, External: {1:.2f} F, Internal: {2:.2f} C, Internal: {3:.2f} F, - Humidity: {4:.2f} % - Pressure: {5:.2f} hPa, - CO2: {6:.2f} ppm, - TVOCs: {7:.2f} ppb".format(External, External2, InternalC, InternalF, Humidity, Pressure, CO2, TVOCs)
        print(output)
        time.sleep(5)
        clear_output()
        time_elapsed = time.time() - time_calculation
        if time_elapsed == 30:
            time_calculation = time.time()
            aio.send_data(co2Feed.key, CO2, metadata)
            aio.send_data(tvocsFeed.key, TVOC, metadata)
            aio.send_data(soundFeed.key, Amps[0], metadata)
            aio.send_data(luxFeed.key, Light, metadata)
            aio.send_data(intTempCfeed.key, InternalC, metadata)
            aio.send_data(intTempFfeed.key, InternalF, metadata)
            aio.send_data(extemp2Feed.key, External2, metadata)
            aio.send_data(extempFeed.key, External, metadata)
            aio.send_data(humidFeed.key, Humidity, metadata)
            aio.send_data(pressFeed.key, Pressure, metadata)
            print("Sending Data to the AdaFruit Cloud......")
    except IOError:
        pass
    except KeyboardInterrupt:
        print("Loop Terminated by User")
    except RuntimeError:
        print("Not Good")